In [4]:
!pip install bs4
!pip install pandas
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd
import pprint

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
#Start with standing to get each team's URL
base_standings = "https://www.espn.com/college-football/standings/_/season/"
year = 2025

In [6]:
#Start with 1 year. Could apply to all years if desired.  
standings_url = base_standings + str(year)

In [7]:
html = urlopen(standings_url)
bs_html = BeautifulSoup(html, 'html.parser')

In [8]:
#Go to standings page. just to check it out.
standings_url

'https://www.espn.com/college-football/standings/_/season/2025'

In [10]:
#Find all table titles.
conference_names = bs_html.find_all('div', {'class':'Table__Title'})

In [11]:
conference = []
teams = []
team_url = []
all_dfs_names = []
for k in conference_names:
    next_div = k.find_next("div")
    tables = next_div.find_all('table')
    names = tables[0]
    headers1 = [th.get_text(strip=True) for th in names.find_all("th")]
    n_cols1 = len(headers1)
    data1 = {h: [] for h in headers1}
    rows1 = names.find_all('tr')
    
    team_links = names.find_all("a")
    
    for j in team_links:
        teams.append(j.text)
        conference.append(k.text)
        team_url.append(j['href'])
        
    table = names
    headers = headers1
    n_cols = len(headers)
    data = {h: [] for h in headers}

    for tr in table.select("tbody tr"):
        cells = tr.find_all(["td", "th"])
    
        for i, header in enumerate(headers):
            value = cells[i].get_text(strip=True) if i < len(cells) else None
            data[header].append(value)
    
    df = pd.DataFrame(data)
    df['conference'] = k.text
    all_dfs_names.append(df)

In [12]:
all_dfs_stats = []
for k in conference_names:
    next_div = k.find_next("div")
    tables = next_div.find_all('table')
    stats = tables[1]
    
    data = []
    for row in stats.find_all("tr"):
        cells = row.find_all("td")   # <-- td only = data rows, no headers
        if cells:
            row_data = [cell.get_text(strip=True) for cell in cells]
            data.append(row_data)
    df = pd.DataFrame(data)
    df['conference'] = k.text
    all_dfs_stats.append(df)

In [15]:
#Combine list of dataframes into 1 dataframe.
combined_df_names = pd.concat(all_dfs_names, ignore_index=True)
combined_df_stats = pd.concat(all_dfs_stats, ignore_index=True)

In [16]:
#Drop missing rows in the names dataset. 
combined_df_names = combined_df_names.dropna()

In [17]:
#Manually assign column names for statistics
cols_stats = ['WL Conf', 'PF conf', 'PA conf', 'WL overall', 'PF overall', 'PA overall', 'home overall', 'away overall', 'streak overall', 'AP Polls', 'USA Polls', 'conference']

In [18]:
#Manually assign column names for names dataset. 
cols_names = ["Team Name", "Conference"]

In [19]:
#Apply column names. 
combined_df_stats.columns = cols_stats
combined_df_names.columns = cols_names

In [22]:
#Reset index. Going to merge by row number. 
combined_df_names = combined_df_names.reset_index()

In [24]:
#Combine the 2 datasets. By row number.
standings_statistics_2025 = pd.concat([combined_df_names, combined_df_stats], axis=1)

In [25]:
#display the dataset.
display(standings_statistics_2025)

,index,Team Name,Conference,WL Conf,PF conf,PA conf,WL overall,PF overall,PA overall,home overall,away overall,streak overall,AP Polls,USA Polls,conference
0,0,11TULNTulane Green Wave,American Conference,7-1,244,167,11-3,388,335,7-0,4-3,L1,2-1,2-1,American Conference
1,2,24UNTNorth Texas Mean Green,American Conference,7-1,382,228,12-2,631,371,6-1,5-1,W1,0-2,1-1,American Conference
2,4,23NAVYNavy Midshipmen,American Conference,7-1,261,209,11-2,409,325,6-0,3-2,W4,1-1,0-1,American Conference
3,6,ECUEast Carolina Pirates,American Conference,6-2,278,183,9-4,425,261,5-1,3-3,W2,0-0,0-0,American Conference
4,8,USFSouth Florida Bulls,American Conference,6-2,389,194,9-4,526,304,6-0,3-3,L1,2-1,2-1,American Conference
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,255,ULLouisiana Ragin' Cajuns,Sun Belt Conference,5-3,238,250,6-7,338,380,4-2,2-4,L1,0-1,0-0,Sun Belt Conference
134,256,ARSTArkansas State Red Wolves,Sun Belt Conference,5-3,200,196,7-6,327,356,3-3,3-3,W2,0-1,0-1,Sun Belt Conference
135,257,TXSTTexas State Bobcats,Sun Belt Conference,3-5,288,267,7-6,474,377,4-2,2-4,W4,0-0,0-1,Sun Belt Conference
136,258,USASouth Alabama Jaguars,Sun Belt Conference,3-5,212,244,4-8,318,365,2-4,2-4,L1,0-1,0-0,Sun Belt Conference


In [26]:
#Add in year column
standings_statistics_2025['year'] = year

In [27]:
#Create the teams links.
teams_information = pd.DataFrame({
    'Team Name': teams,
    'Conference': conference,
    'Team URL': team_url
    })

In [28]:
teams_information = teams_information.iloc[2::3]

In [29]:
teams_information = teams_information.reset_index()

In [31]:
display(teams_information)

,index,Team Name,Conference,Team URL
0,2,Tulane Green Wave,American Conference,/college-football/team/_/id/2655/tulane-green-...
1,5,North Texas Mean Green,American Conference,/college-football/team/_/id/249/north-texas-me...
2,8,Navy Midshipmen,American Conference,/college-football/team/_/id/2426/navy-midshipmen
3,11,East Carolina Pirates,American Conference,/college-football/team/_/id/151/east-carolina-...
4,14,South Florida Bulls,American Conference,/college-football/team/_/id/58/south-florida-b...
...,...,...,...,...
131,395,Louisiana Ragin' Cajuns,Sun Belt Conference,/college-football/team/_/id/309/louisiana-ragi...
132,398,Arkansas State Red Wolves,Sun Belt Conference,/college-football/team/_/id/2032/arkansas-stat...
133,401,Texas State Bobcats,Sun Belt Conference,/college-football/team/_/id/326/texas-state-bo...
134,404,South Alabama Jaguars,Sun Belt Conference,/college-football/team/_/id/6/south-alabama-ja...


In [32]:
#Create team full URL
team_full_url = []
for i in teams_information['Team URL']:
    team_full_url.append("https://www.espn.com" + i)

In [33]:
teams_information['Team_Full_URL'] = team_full_url

In [34]:
#Create stats URLs 
stats_url = []
for i in teams_information['Team_Full_URL']:
    dropped = i.rsplit('/', 1)[0]
    stat_url = dropped.replace("team/", "team/stats/")
    stat_url = stat_url + "/season/" + str(year)
    stats_url.append(stat_url)
    

In [36]:
teams_information['stats_urls'] = stats_url
teams_information["team_stats_urls"] = teams_information["stats_urls"].str.replace("team/stats/_/", "team/stats/_/type/team/", regex=False)#https://www.espn.com/college-football/team/stats/_/id/2426/season/2025

In [38]:
#Display dataframe teams information
display(teams_information)

,index,Team Name,Conference,Team URL,Team_Full_URL,stats_urls,team_stats_urls
0,2,Tulane Green Wave,American Conference,/college-football/team/_/id/2655/tulane-green-...,https://www.espn.com/college-football/team/_/i...,https://www.espn.com/college-football/team/sta...,https://www.espn.com/college-football/team/sta...
1,5,North Texas Mean Green,American Conference,/college-football/team/_/id/249/north-texas-me...,https://www.espn.com/college-football/team/_/i...,https://www.espn.com/college-football/team/sta...,https://www.espn.com/college-football/team/sta...
2,8,Navy Midshipmen,American Conference,/college-football/team/_/id/2426/navy-midshipmen,https://www.espn.com/college-football/team/_/i...,https://www.espn.com/college-football/team/sta...,https://www.espn.com/college-football/team/sta...
3,11,East Carolina Pirates,American Conference,/college-football/team/_/id/151/east-carolina-...,https://www.espn.com/college-football/team/_/i...,https://www.espn.com/college-football/team/sta...,https://www.espn.com/college-football/team/sta...
4,14,South Florida Bulls,American Conference,/college-football/team/_/id/58/south-florida-b...,https://www.espn.com/college-football/team/_/i...,https://www.espn.com/college-football/team/sta...,https://www.espn.com/college-football/team/sta...
...,...,...,...,...,...,...,...
131,395,Louisiana Ragin' Cajuns,Sun Belt Conference,/college-football/team/_/id/309/louisiana-ragi...,https://www.espn.com/college-football/team/_/i...,https://www.espn.com/college-football/team/sta...,https://www.espn.com/college-football/team/sta...
132,398,Arkansas State Red Wolves,Sun Belt Conference,/college-football/team/_/id/2032/arkansas-stat...,https://www.espn.com/college-football/team/_/i...,https://www.espn.com/college-football/team/sta...,https://www.espn.com/college-football/team/sta...
133,401,Texas State Bobcats,Sun Belt Conference,/college-football/team/_/id/326/texas-state-bo...,https://www.espn.com/college-football/team/_/i...,https://www.espn.com/college-football/team/sta...,https://www.espn.com/college-football/team/sta...
134,404,South Alabama Jaguars,Sun Belt Conference,/college-football/team/_/id/6/south-alabama-ja...,https://www.espn.com/college-football/team/_/i...,https://www.espn.com/college-football/team/sta...,https://www.espn.com/college-football/team/sta...


In [40]:
#Now go to the stats. and collect the data. 
all_dfs_names = []
all_dfs_stats = []
school_name_list = []
school_name_year_list = []
passing = []
rushing = []
receiving = []
tackles = []
interceptions = []
record_list = []
urls_list = []
import time
for i in teams_information['stats_urls']:
    html = urlopen(i)
    bs_html = BeautifulSoup(html, 'html.parser')
    time.sleep(1)
    school_names = bs_html.find_all("h1")
    school_name = school_names[0].text
    school_name_year = school_names[1].text
    record = bs_html.find('ul', {'class':'list flex ClubhouseHeader__Record n8 ml4'})
    record = record.text
    try:
        team_leaders = bs_html.find('div', {'id':'a11yLeadersTitle'})
        next_div = team_leaders.find_next("div")
        stats_indiv = next_div.find_all("a")
        passing.append(stats_indiv[0].text)
        rushing.append(stats_indiv[1].text)
        receiving.append(stats_indiv[2].text)
        tackles.append(stats_indiv[3].text)
        interceptions.append(stats_indiv[4].text)
        school_name_year_list.append(school_name_year)
        record_list.append(record)
        urls_list.append(i)
    except:
        passing.append("missing")
        rushing.append("missing")
        receiving.append("missing")
        tackles.append("missing")
        interceptions.append("missing")
        school_name_year_list.append(school_name_year)
        record_list.append(record)
        urls_list.append(i)
    stats_sections = bs_html.find_all('div', {'class':'Table__Title'})
    next_div = stats_sections[1].find_next("div")
    for i in stats_sections:
        school_names = bs_html.find_all("h1")
        school_name = school_names[0].text
        school_name_year = school_names[1].text
        record = bs_html.find('ul', {'class':'list flex ClubhouseHeader__Record n8 ml4'})
        record = record.text
        stats = i.find_next("div")
        tables = stats.find_all('table')
        names = tables[0]
        data_stats = tables[1]
        headers1 = [th.get_text(strip=True) for th in names.find_all("th")]
        headers2 = [th.get_text(strip=True) for th in data_stats.find_all("th")]
        n_cols1 = len(headers1)
        n_cols2 = len(headers2)
        data1 = {h: [] for h in headers1}
        data2 = {h: [] for h in headers2}
        rows1 = names.find_all('tr')
        rows2 = data_stats.find_all('tr')
        table = data_stats
        headers = headers2
        n_cols = len(headers)
        data = {h: [] for h in headers}
    
        for tr in table.select("tbody tr"):
            cells = tr.find_all(["td", "th"])
        
            for i, header in enumerate(headers):
                value = cells[i].get_text(strip=True) if i < len(cells) else None
                data[header].append(value)
        
        df = pd.DataFrame(data)
        df['school_name'] = school_name
        df['school_year_name'] = school_name_year
        df['record'] = record
        all_dfs_names.append(df)
        
        table = names
        headers = headers1
        n_cols = len(headers)
        data = {h: [] for h in headers}
    
        for tr in table.select("tbody tr"):
            cells = tr.find_all(["td", "th"])
        
            for i, header in enumerate(headers):
                value = cells[i].get_text(strip=True) if i < len(cells) else None
                data[header].append(value)
        
        df2 = pd.DataFrame(data)
        df2['school_name'] = school_name
        df2['school_year_name'] = school_name_year
        df2['record'] = record
        all_dfs_stats.append(df2)

In [43]:
#Ranges
passing_index = list(range(0, len(all_dfs_names), 5))
rushing_index = list(range(1, len(all_dfs_names), 5))
receiving_index = list(range(2, len(all_dfs_names), 5))
kicking_index = list(range(3, len(all_dfs_names), 5))
defense_index = list(range(4, len(all_dfs_names), 5))

In [44]:
selected_dfs_passing = [all_dfs_names[i] for i in passing_index]
selected_dfs_rushing = [all_dfs_names[i] for i in rushing_index]
selected_dfs_receiving = [all_dfs_names[i] for i in receiving_index]
selected_dfs_kicking = [all_dfs_names[i] for i in kicking_index]
selected_dfs_defense = [all_dfs_names[i] for i in defense_index]

In [45]:
selected_dfs_passing_stats = [all_dfs_stats[i] for i in passing_index]
selected_dfs_rushing_stats = [all_dfs_stats[i] for i in rushing_index]
selected_dfs_receiving_stats = [all_dfs_stats[i] for i in receiving_index]
selected_dfs_kicking_stats = [all_dfs_stats[i] for i in kicking_index]
selected_dfs_defense_stats = [all_dfs_stats[i] for i in defense_index]

In [46]:
final_df_passing_names = pd.concat(selected_dfs_passing, ignore_index=True)
final_df_rushing_names = pd.concat(selected_dfs_rushing, ignore_index=True)
final_df_receiving_names = pd.concat(selected_dfs_receiving, ignore_index=True)
final_df_kicking_names = pd.concat(selected_dfs_kicking, ignore_index=True)
final_df_defense_names = pd.concat(selected_dfs_defense, ignore_index=True)
final_df_passing_stats = pd.concat(selected_dfs_passing_stats, ignore_index=True)
final_df_rushing_stats = pd.concat(selected_dfs_rushing_stats, ignore_index=True)
final_df_receiving_stats = pd.concat(selected_dfs_receiving_stats, ignore_index=True)
final_df_kicking_stats = pd.concat(selected_dfs_kicking_stats, ignore_index=True)
final_df_defense_stats = pd.concat(selected_dfs_defense_stats, ignore_index=True)

In [47]:
#Then combine. 
df_passing = pd.concat([final_df_passing_names, final_df_passing_stats], axis=1)
df_rushing = pd.concat([final_df_rushing_names, final_df_rushing_stats], axis=1)
df_receiving = pd.concat([final_df_receiving_names, final_df_receiving_stats], axis=1)
df_kicking = pd.concat([final_df_kicking_names, final_df_kicking_stats], axis=1)
df_defense = pd.concat([final_df_defense_names, final_df_defense_stats], axis=1)

In [48]:
#Set location to export datasets. This path will be different for you. 
import os
os.chdir("Z:\\jrg363\\Workshops SP26\\WSP")

In [49]:
df_passing.to_csv("Passing_stats.csv")
df_rushing.to_csv("Rushing_stats.csv")
df_receiving.to_csv("Receiving_stats.csv")
df_kicking.to_csv("kicking_stats.csv")
df_defense.to_csv("defense_stats.csv")
leaders.to_csv("team_leaders.csv")

In [50]:
#Now just need team statistics. 
team_stats_dfs_list_names = []
team_stats_dfs_list_stats = []
for i in teams_information["team_stats_urls"]:
    html = urlopen(i)
    bs_html = BeautifulSoup(html, 'html.parser')
    team_name = bs_html.find_all('h1')
    team_name = team_name[1]
    table_sections = bs_html.find('div', {'class':'Table__Title'})  
    next_div = table_sections.find_next("div")
    tables = next_div.find_all('table')
    names = tables[0]
    data_stats = tables[1]
    headers1 = [th.get_text(strip=True) for th in names.find_all("th")]
    headers2 = [th.get_text(strip=True) for th in data_stats.find_all("th")]
    n_cols1 = len(headers1)
    n_cols2 = len(headers2)
    data1 = {h: [] for h in headers1}
    data2 = {h: [] for h in headers2}
    rows1 = names.find_all('tr')
    rows2 = data_stats.find_all('tr')
    table = data_stats
    headers = headers2
    n_cols = len(headers)
    data = {h: [] for h in headers}

    for tr in table.select("tbody tr"):
        cells = tr.find_all(["td", "th"])
    
        for i, header in enumerate(headers):
            value = cells[i].get_text(strip=True) if i < len(cells) else None
            data[header].append(value)
    
    df = pd.DataFrame(data)
    df.rename(columns={df.columns[0]: "stats"}, inplace=True)
    df['school_name'] = team_name.text
    team_stats_dfs_list_stats.append(df)
    
    table = names
    headers = headers1
    n_cols = len(headers)
    data = {h: [] for h in headers}

    for tr in table.select("tbody tr"):
        cells = tr.find_all(["td", "th"])
    
        for i, header in enumerate(headers):
            value = cells[i].get_text(strip=True) if i < len(cells) else None
            data[header].append(value)
    
    df2 = pd.DataFrame(data)
    df2['school_name'] = team_name.text
    team_stats_dfs_list_names.append(df2)

In [51]:
team_stats_names = pd.concat(team_stats_dfs_list_names, ignore_index=True)
team_stats_stats = pd.concat(team_stats_dfs_list_stats, ignore_index=True)

In [52]:
#Now merge by row number
df_team_stats_all = pd.concat([team_stats_names.reset_index(drop=True),
                       team_stats_stats.reset_index(drop=True)],
                      axis=1)

In [53]:
display(df_team_stats_all)

,Scoring,school_name,stats,school_name
0,Total Points Per Game,Tulane Green Wave Team Stats 2025,27.7,Tulane Green Wave Team Stats 2025
1,Total Points,Tulane Green Wave Team Stats 2025,388,Tulane Green Wave Team Stats 2025
2,Total Touchdowns,Tulane Green Wave Team Stats 2025,45,Tulane Green Wave Team Stats 2025
3,1st Downs,Tulane Green Wave Team Stats 2025,,Tulane Green Wave Team Stats 2025
4,Total 1st downs,Tulane Green Wave Team Stats 2025,307,Tulane Green Wave Team Stats 2025
...,...,...,...,...
6795,Time of Possession,UL Monroe Warhawks Team Stats 2025,,UL Monroe Warhawks Team Stats 2025
6796,Possession Time Seconds,UL Monroe Warhawks Team Stats 2025,29:37,UL Monroe Warhawks Team Stats 2025
6797,Miscellaneous,UL Monroe Warhawks Team Stats 2025,,UL Monroe Warhawks Team Stats 2025
6798,Fumbles-Lost,UL Monroe Warhawks Team Stats 2025,12-7,UL Monroe Warhawks Team Stats 2025


In [54]:
df_team_stats_all.to_csv("Team_Stats_All_2025.csv")